In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_excel("../datasets/msft_test.xlsx")

df


,Global Company Key,S&P Domestic Long Term Issuer Credit Rating,S&P Domestic Short Term Issuer Credit Rating,Data Date,S&P Quality Ranking - Current,Company Name,Ticker Symbol
0,12141,AAA,A-1+,20100131,A-,MICROSOFT CORP,MSFT
1,12141,AAA,A-1+,20100228,A-,MICROSOFT CORP,MSFT
2,12141,AAA,A-1+,20100331,A-,MICROSOFT CORP,MSFT
3,12141,AAA,A-1+,20100430,A-,MICROSOFT CORP,MSFT
4,12141,AAA,A-1+,20100531,A-,MICROSOFT CORP,MSFT
5,12141,AAA,A-1+,20100630,A-,MICROSOFT CORP,MSFT
6,12141,AAA,A-1+,20100731,A-,MICROSOFT CORP,MSFT
7,12141,AAA,A-1+,20100831,A-,MICROSOFT CORP,MSFT
8,12141,AAA,A-1+,20100930,A-,MICROSOFT CORP,MSFT
9,12141,AAA,A-1+,20101031,A-,MICROSOFT CORP,MSFT
